# Tester notebook

This notebook aims to create enviroment to simply test strategies developed in Stocks & Loss system for python

In [1]:
# imports 
from scraper import stock_daily
from analyzer import Analyzer
import numpy as np
import pandas as pd
from IPython.display import display

## Select stocks

In [4]:
xls = pd.ExcelFile('TickerPerSector.xls')
fundTickers = pd.DataFrame()
Tickers = pd.DataFrame()
fundTickers["Fund/sector name"] = xls.sheet_names
# Tickers = pd.DataFrame(,columns=funTickers["Index fund name"])
for i in range(len(xls.sheet_names)):
    sheet1 = pd.read_excel(xls, i)
    fundTickers.loc[i,["Index fund ticker"]] = sheet1.columns[1]
    Tickers[fundTickers.loc[i,["Fund/sector name"]]] = sheet1[sheet1.columns[1]][1:]
pd.set_option('display.expand_frame_repr', False)    
print(Tickers)

ValueError: Must have equal len keys and value when setting with an iterable

In [ ]:
ticker_name = "XELB"

In [ ]:
anal = Analyzer(ticker=ticker_name,data=stock_daily(ticker_name, save=False).data)

anal.strategy(buyStrategy=['Mcstoch_ut1','Mcstoch_ut2', 'Mcstoch_ut3', 'Mcstoch_ut4'],sellStrategy=['Mcstoch'],stopLoss=True,stopLossValue=0.05,profitTaker=True,profitTakerValue=0.1)
tradeSummary = anal.profit(capitalForEachTrade=400,comission=2)

In [ ]:
profitAbsolute = tradeSummary["Profit[$]"].sum()
profitRelative = tradeSummary["Profit[%]"].sum()
profitByHolding = 100*((anal.data["Close"].iloc[-1]-anal.data["Close"].iloc[0])/anal.data["Close"].iloc[0])

print('Absolute profit during last year: ',np.round(profitAbsolute,2),'$')
print('Relative profit during last year: ',np.round(profitRelative,2),'%')
print('Relative profit by holding during last year: ',np.round(profitByHolding,2),'%')

Absolute profit during last year:  0.48 $
Relative profit during last year:  0.12 %
Relative profit by holding during last year:  147.75 %


In [ ]:
display(tradeSummary)

,Buy date,Buy price,Buy value,Position,Sell date,Sell price,Sell value,Comission,Good trade?,Profit[%],Profit[$]
0,2020-08-25,0.8404,400,475.963838,2020-08-26,0.809600,385.340321,2,0.0,-4.164920,-16.659679
1,2020-10-15,0.8711,400,459.189526,2020-10-16,0.958210,440.000000,2,1.0,9.500000,38.000000
2,2020-10-30,0.8896,400,449.640298,2020-11-02,0.845120,380.000000,2,0.0,-5.500000,-22.000000
3,2020-11-03,0.8900,400,449.438209,2020-11-04,0.845500,380.000000,2,0.0,-5.500000,-22.000000
4,2020-11-05,0.9499,400,421.096970,2020-11-09,0.902405,380.000000,2,0.0,-5.500000,-22.000000
5,2020-11-18,1.1700,400,341.880354,2020-11-19,1.111500,380.000000,2,0.0,-5.500000,-22.000000
6,2020-12-04,1.0900,400,366.972466,2020-12-07,1.035500,380.000000,2,0.0,-5.500000,-22.000000
7,2020-12-09,1.0300,400,388.349525,2020-12-10,1.133000,440.000000,2,1.0,9.500000,38.000000
8,2020-12-14,1.1100,400,360.360356,2020-12-15,1.221000,440.000000,2,1.0,9.500000,38.000000
9,2020-12-24,1.1400,400,350.877197,2020-12-29,1.254000,440.000000,2,1.0,9.500000,38.000000
